# What is Julia?

## "A New Approach to Technical Computing"

- Less than 10 years old - project started in 2012
- Growing use in scientific computing, data science, and machine learning
- Users in industry, government, and academia, all over the world

- One-sentence pitch: code as fast to write as Python, as fast to run as C++
- Actually, Julia can often be faster than C++
- Much easier to parallelize code across multiple cores, multiple nodes, or the GPU than Python
- Code is readable, easy to share with students, collaborators, and the public
- First class support for linear algebra and numerical computing
- The community is friendly and full of expertise from many domains
- Easy to learn if you already know Python
- Very good support for testing, documenting, and benchmarking code
- **Julia is one-based indexed, this isn't changing, love it or leave it.**

## Success in a variety of fields

### Physics
- A group in CCQ, including Matt Fishman, Miles Stoudenmire, and me, has rewritten the C++ ITensors library in Julia and seen equivalent performance while adding features that would have been much harder to implement in C++ (like GPU support)
- Yao.jl, a state of the art quantum circuit simulator
- DiffEqPhysics


### Astronomy
- Celeste.jl
- Eric Agol's group in exoplanet astronomy
- https://github.com/marius311
- https://github.com/JuliaAstro/Cosmology.jl
- https://juliaastro.github.io/SkyCoords.jl/latest/#Performance-1
- https://github.com/JuliaAstro/LombScargle.jl#performance
- https://github.com/JuliaAstro/Photometry.jl

### Biology
- BioJulia packages
- BioSequences.jl performance blogpost (https://biojulia.net/post/seq-lang/)
- DiffEqBiological

### Climate Science
- Clima.jl
- Oceananigans


## Drawbacks of using Julia

- Since the language is new, many people are unfamiliar with it and unwilling to switch
- Less well developed community than Python has - no "JuliaLadies", for example, or regional JuliaCons
- Onboarding and tutorial materials still lacking - a great opportunity for someone to get involved
- Fewer mature packages than Python or C++
- Tooling is less mature -- debuggers, IDEs, etc (this is improving quickly)

## Just how fast is this language?

We can look at two kinds of benchmarks: "micro", for small operations that are repeated many times (like `sqrt`) and "macro", for a complete simulation/data analysis.

![micro-benchmarks](https://julialang.org/assets/benchmarks/benchmarks.svg)

## It's easy to use libraries written in other languages

This way, you can use Julia for parts of the simulation you want to rewrite now, while still retaining the battle-tested library you've put many hours into developing.

### From Python

Julia has a very nice package called `PyCall` that lets you import Python libraries, use Python types, and call Python functions. Let's look at an example:

In [ ]:
using PyCall, Conda
Conda.add("geopy", channel="conda-forge")
gp = pyimport("geopy")
using JLD, FileIO
coords = jldopen("coord_df.jld", "r") do file
    read(file, "coords")
end
gcs = pyimport("geopy")."geocoders"
locator = gcs.Nominatim(user_agent="citibike", timeout=nothing)
gx = pyimport("geopy.extra.rate_limiter")
rl = gx.RateLimiter(locator.reverse, min_delay_seconds=1.1, max_retries=4)

located = coords |> Tables.transform(Dict(:coord=>(x->rl(x)))) |> DataFrame

### From C

- Calling C from Julia (or Julia from C!) is natively supported, as is using C `struct`s, and all those good things. Julia and C can pass arrays back and forth, and it's simple to call any function in a shared object library from Julia.

### From FORTRAN

## Showcases

In this section we'll see some cool examples of things Julia and its packages are able to do:
- Random walks
- Interactive plotting
  - Changing contour plot?
- Multi-node job handling
  - `ClusterManagers.jl`
  - `WorkerGroup`
- Parallelism
  - Threads - show atomics, maybe a lock?
  - `pmap` - 2D array example
- Differential equations solving
  - Neural ODE solver
  - https://github.com/SciML/DiffEqTutorials.jl
- Native Julia machine learning
  - Flux
  - Transformers
- Handling big datasets
  - Citibike

## Resources to learn more

- There is a Julia book!
- Julia slack is very welcoming and we have dedicated channels for physics, astronomy, biology, and areas like HPC
- We also have a discourse forum for people to ask questions and get help
- The Julia youtube page has many talks from previous JuliaCons, which often include Jupyter notebook tutorials
- Learn X in Y minutes - a basic introduction to Julia syntax